# Arctic sea ice extent prediction

**Objective:** Predict arctic sea ice extent one month in advance using ML forecasting.

**Reference:** https://www.cienciadedatos.net/documentos/py27-time-series-forecasting-python-scikitlearn.html

In [2]:
import numpy as np
import pandas as pd
import datascience as ds

### 1 Data pre-processing

Load data, prepare data for prediction one month in advance. Compression is not used because dataset it relatively small and does not require it. 

In [ ]:
freq = "1M"

In [ ]:
data_mean = pd.read_csv("Data/Arctic_domain_mean_1979_2018.csv").rename(columns={"Day": "date"}).drop(columns=["Year"])
data_mean["date"] = pd.to_datetime(data_mean['date'], infer_datetime_format=True)
data_mean = data_mean.set_index("date")
data_mean = data_mean.asfreq(freq)

data_mean_monthly = pd.read_csv("Data/Arctic_domain_mean_monthly_1979_2018.csv")

In [21]:
data_mean.head()

,wind_10m,specific_humidity,LW_down,SW_down,rainfall,snowfall,sosaline,sst,t2m,surface_pressure,sea_ice_extent
date,,,,,,,,,,,
1979-01-07,5.303105,0.945889,187.574231,1.480709,1.111009,0.980082,33.341556,273.410304,250.509781,979.765760,14928584
1979-01-14,5.674731,0.862226,193.242351,2.401907,1.288898,1.103170,33.341556,273.373557,251.998659,985.454076,15352126
1979-01-21,5.128323,0.824095,187.889395,3.283766,1.013956,0.866181,33.341556,273.365295,250.480041,988.920873,15709302
1979-01-28,5.029017,0.717743,179.309929,6.361050,0.919005,0.859442,33.341556,273.268675,248.570151,991.017959,15776769
1979-02-04,5.018918,0.522890,161.390858,9.912303,0.622876,0.574732,33.365164,273.168560,244.170769,987.778436,15838632


Normalize data (normalization is used instead of standardization due to non-normal feature distributions).

In [22]:
from sklearn.preprocessing import MinMaxScaler

min_max_scaler = MinMaxScaler()
data_mean.iloc[:,0:10] = data_mean.iloc[:,0:10] = min_max_scaler.fit_transform(data_mean.iloc[:,0:10])

In [23]:
data_mean.head()

,wind_10m,specific_humidity,LW_down,SW_down,rainfall,snowfall,sosaline,sst,t2m,surface_pressure,sea_ice_extent
date,,,,,,,,,,,
1979-01-07,0.474241,0.081608,0.166309,0.002855,0.273066,0.505817,0.613641,0.070796,0.169822,0.464489,14928584
1979-01-14,0.577811,0.065467,0.202311,0.006344,0.343578,0.571296,0.613641,0.064814,0.209709,0.623994,15352126
1979-01-21,0.425531,0.058111,0.168310,0.009684,0.234597,0.445225,0.613641,0.063469,0.169025,0.721205,15709302
1979-01-28,0.397855,0.037593,0.113816,0.021340,0.196961,0.441640,0.613641,0.047740,0.117859,0.780009,15776769
1979-02-04,0.395040,0.000000,0.000000,0.034791,0.079581,0.290184,0.659927,0.031441,0.000000,0.689170,15838632


### 2 Split data

Data was split into 60/30/30 train/validation/test. The datapoints were not randomized because model is a time series. 

| Category | Specifications | 
| - | - |
| Training data | First 60% of data (288 entries)|
| Validation data | Middle 20% of data (96 entries)|
| Testing data | Last 20% of data (96 entries)|

In [24]:
split = [0.6, 0.2, 0.2]

train_entries = int(len(data_mean) * split[0])
val_entries = int(len(data_mean) * split[1])
test_entries = len(data_mean) - train_entries - val_entries

train_M = data_mean.iloc[0:train_entries]
val_M = data_mean.iloc[train_entries:-test_entries]
test_M  = data_mean.iloc[-test_entries:]

In [25]:
print(
    len(train_M),
    len(val_M),
    len(test_M)
)

1252 417 418


### 4 Linear regression

Forecast sea ice extent using sklearn forecasting.

In [26]:
from sklearn.linear_model import LinearRegression
from sklearn.linear_model import Lasso
from sklearn.ensemble import RandomForestRegressor
from sklearn.metrics import mean_squared_error
from sklearn.preprocessing import StandardScaler
from sklearn.preprocessing import MinMaxScaler
from sklearn.pipeline import make_pipeline

from skforecast.ForecasterAutoreg import ForecasterAutoreg
from skforecast.ForecasterAutoregCustom import ForecasterAutoregCustom
from skforecast.ForecasterAutoregMultiOutput import ForecasterAutoregMultiOutput
from skforecast.model_selection import grid_search_forecaster
from skforecast.model_selection import backtesting_forecaster

from sklearn.ensemble import GradientBoostingRegressor

In [27]:
fcast = ForecasterAutoregMultiOutput(
    regressor = RandomForestRegressor(random_state=1),
    steps     = val_entries,
    lags      = 1
)

fcast

ForecasterAutoregMultiOutput 
Regressor: RandomForestRegressor(random_state=1) 
Lags: [1] 
Window size: 1 
Maximum steps predicted: 417 
Included exogenous: False 
Type of exogenous variable: None 
Exogenous variables names: None 
Training range: None 
Training index type: None 
Training index frequency: None 
Regressor parameters: {'bootstrap': True, 'ccp_alpha': 0.0, 'criterion': 'squared_error', 'max_depth': None, 'max_features': 1.0, 'max_leaf_nodes': None, 'max_samples': None, 'min_impurity_decrease': 0.0, 'min_samples_leaf': 1, 'min_samples_split': 2, 'min_weight_fraction_leaf': 0.0, 'n_estimators': 100, 'n_jobs': None, 'oob_score': False, 'random_state': 1, 'verbose': 0, 'warm_start': False} 
Creation date: 2022-10-03 21:02:49 
Last fit date: None 
Skforecast version: 0.4.3 

In [11]:
fcast.fit(y=train_M["sea_ice_extent"])

Optimize hyperparameters.

In [28]:
lags_grid = [20, 50]
param_grid = {'n_estimators': [100, 500],
              'max_depth': [3, 5, 10]}

results_grid = grid_search_forecaster(
    forecaster         = fcast,
    y                  = train_M["sea_ice_extent"],
    param_grid         = param_grid,
    lags_grid          = lags_grid,
    steps              = val_entries,
    refit              = True,
    metric             = "mean_squared_error",
    initial_train_size = int(len(train_M)*0.6),
    fixed_train_size   = False,
    return_best        = True,
    verbose            = False
)

Number of models compared: 12


loop lags_grid: 100%|█████████████████████████████████| 2/2 [12:21:17<00:00, 22238.84s/it]


`Forecaster` refitted using the best-found lags and parameters, and the whole data set: 
  Lags: [ 1  2  3  4  5  6  7  8  9 10 11 12 13 14 15 16 17 18 19 20 21 22 23 24 25
 26 27 28 29 30 31 32 33 34 35 36 37 38 39 40 41 42 43 44 45 46 47 48 49 50] 
  Parameters: {'max_depth': 10, 'n_estimators': 500}
  Backtesting metric: 372952285352.6685



In [31]:
results_grid.to_csv("model_1_results.csv")

In [32]:
import pickle
import os

In [33]:
saved_model = pickle.dump(fcast, open("model_2.pkl", "wb"))

In [ ]:
fcast = pickle.load(open("model_2.pkl", "rb"))

In [ ]:
val_pred = fcast.predict(steps=val_entries)
val_pred.to_csv("val_predictions.csv")
val_pred.head(5)

ERROR! Session/line number was not unique in database. History logging moved to new session 424


2003-01-05    1.382975e+07
2003-01-12    1.435674e+07
2003-01-19    1.469749e+07
2003-01-26    1.491730e+07
2003-02-02    1.505359e+07
Freq: W-SUN, Name: pred, dtype: float64

### Chart prediction

Graph prediction against real values to visualize performance.

In [35]:
import matplotlib.pyplot as plt
%matplotlib inline

In [43]:
fig, ax = plt.subplots(figsize=(12, 4), dpi=1200)
train_M["sea_ice_extent"].plot(ax=ax, label='train')
val_M["sea_ice_extent"].plot(ax=ax, label='validation')
test_M["sea_ice_extent"].plot(ax=ax, label='test')
val_pred.plot(ax=ax, label='predictions')
ax.legend()
plt.ylabel("Sea ice extent (in millions of miles)")
plt.xlabel("Year")
plt.title("[W] Sea Ice Extent Prediction")
plt.plot()
plt.savefig("model_2_graph.png")

### 5 Evaluate model

Generate Epoch v. MSE table to chart how performance increases (measure by decrease in error) which each iterative learning cycle.

In [45]:
from sklearn.metrics import mean_squared_error
val_mse = mean_squared_error(val_M["sea_ice_extent"], val_pred, squared=False)
print(val_M["sea_ice_extent"][70]/1000)
print(val_pred[70]/1000)
val_mse

12796.198
13791.7346374


968884.22559705377

In [44]:
from sklearn.metrics import r2_score
val_r2 = r2_score(val_M["sea_ice_extent"], val_pred)
val_r2

0.91562958988367738